# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [19]:
%reload_ext dotenv
%dotenv

In [20]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [21]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
parquet_files = [x for x in parquet_files if os.path.isfile(x)]

In [22]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532215,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206856,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712814,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276863,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299919,2819626,Health Care,Life Sciences Tools & Services,2000
...,...,...,...,...,...,...,...,...,...,...
ZTS,2024-07-01,173.259995,175.460007,172.000000,172.580002,172.580002,1589800,Health Care,Pharmaceuticals,2024
ZTS,2024-07-02,172.490005,175.990005,172.490005,174.809998,174.809998,2454200,Health Care,Pharmaceuticals,2024
ZTS,2024-07-03,175.440002,177.669998,175.259995,175.600006,175.600006,1251800,Health Care,Pharmaceuticals,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [23]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False
    ).apply(lambda x: x.assign(Close_lag = x['Close'].shift(1))))

dd_feat = (dd_feat.groupby('ticker', group_keys=False
     ).apply(lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1))
     ).assign(returns = lambda x: x['Adj Close']/x['Adj_Close_lag']-1
     ).assign(hi_lo_range = lambda x: x['High']-x['Low']))

C:\Users\koush\AppData\Local\Temp\ipykernel_14172\572876171.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False
C:\Users\koush\AppData\Local\Temp\ipykernel_14172\572876171.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_feat.groupby('ticker', group_keys=False


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [24]:
df = dd_feat.compute()
df.groupby('ticker', group_keys=False).apply(lambda x: x.assign(roll_avg_ret = x['returns'].rolling(10).mean()))

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,roll_avg_ret
ticker,,,,,,,,,,,,,,,
IP,2004-01-02,40.671135,40.671135,39.858646,39.914680,18.959656,2145530,Materials,Paper & Plastic Packaging Products & Materials,2004,NaN,NaN,NaN,0.812489,NaN
IP,2004-01-05,40.110798,40.736507,39.914680,40.717831,19.341162,2752236,Materials,Paper & Plastic Packaging Products & Materials,2004,39.914680,18.959656,0.020122,0.821827,NaN
IP,2004-01-06,40.503033,41.016678,40.409645,40.876591,19.416573,2976458,Materials,Paper & Plastic Packaging Products & Materials,2004,40.717831,19.341162,0.003899,0.607033,NaN
IP,2004-01-07,40.633778,41.520981,40.559067,41.334202,19.633938,3738642,Materials,Paper & Plastic Packaging Products & Materials,2004,40.876591,19.416573,0.011195,0.961914,NaN
IP,2004-01-08,41.334202,42.034622,41.231472,42.006603,19.953339,4110097,Materials,Paper & Plastic Packaging Products & Materials,2004,41.334202,19.633938,0.016268,0.803150,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BXP,2008-12-24,53.490002,54.060001,52.419998,53.340000,29.705631,586700,Real Estate,Office REITs,2008,53.330002,29.700066,0.000187,1.640003,-0.008397
BXP,2008-12-26,54.180000,54.889999,52.270000,53.830002,29.978521,699300,Real Estate,Office REITs,2008,53.340000,29.705631,0.009186,2.619999,0.007463
BXP,2008-12-29,52.849998,52.849998,49.000000,50.410000,28.433067,1924700,Real Estate,Office REITs,2008,53.830002,29.978521,-0.051552,3.849998,-0.007238


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

COMMENT:
It is not necessary to convert to calculate moving average return.
The choice to use Pands or Dask depends in the size of the data, and the considered task.
Pandas loads data into memory, which could be limiting with bigger datasets causing performance issues.
Dask has similar APIs to Pandas, and efficient at parallel processing potentially speeding up processing.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.